# Redução de Lojas Com Ruptura

O modelo visa reduzir a quantidade de lojas com possíveis rupturas

## Importando modulos

In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo

## Lendo inputs

In [2]:
input_file = "../data/data_smart_execution.xlsx"

### Lojas

In [3]:
sheet_lojas = pd.read_excel(input_file, sheet_name="LOJA")
sheet_lojas['LOJA'].head()

0    A
1    B
2    C
3    D
4    E
Name: LOJA, dtype: object

### Disponibilidade de SKUs

In [4]:
sheet_disponibilidade_sku = pd.read_excel(input_file, sheet_name="DISPONIBILIDADE_SKU")
sheet_disponibilidade_sku.head()

,SKU,QUANTIDADE_DISPONIVEL
0,Mimmo Plus Suave,250
1,Report Premium,150


### Lojas x SKUs

In [5]:
sheet_loja_x_sku = pd.read_excel(input_file, sheet_name="LOJA_SKU")
sheet_loja_x_sku.head()

,LOJA,SKU,CAPACIDADE_MAXIMA,PROBABILIDADE_ACEITAVEL
0,A,Mimmo Plus Suave,53,0.33
1,C,Mimmo Plus Suave,80,0.40
2,D,Mimmo Plus Suave,20,0.11
3,E,Mimmo Plus Suave,55,0.49
4,F,Mimmo Plus Suave,40,0.32


### Probabilidade de Ruptura do SKU na Loja

In [6]:
sheet_prob_ruptura = pd.read_excel(input_file, sheet_name="PROB_RUPTURA")
sheet_prob_ruptura.head()

,LOJA,SKU,NIVEL_RUPTURA,QUANTIDADE_ESTOCADA,PROBABILIDADE_DE_RUPTURA
0,A,Mimmo Plus Suave,1,5,0.92
1,A,Mimmo Plus Suave,2,13,0.81
2,A,Mimmo Plus Suave,3,26,0.62
3,A,Mimmo Plus Suave,4,39,0.32
4,A,Mimmo Plus Suave,5,50,0.22


## Modelo


In [7]:
model = pyo.ConcreteModel()

### Sets

In [8]:
lista_lojas = sheet_lojas["LOJA"].unique()
model.lojas = pyo.Set(initialize=lista_lojas, doc="Lista de lojas")

In [9]:
lista_skus = sheet_disponibilidade_sku["SKU"].unique()
model.skus = pyo.Set(initialize=lista_skus, doc="Lista de SKUs")

In [10]:
lista_loja_x_sku = [
    (row["LOJA"], row["SKU"]) for index, row in sheet_loja_x_sku.iterrows()
]
model.loja_x_sku = pyo.Set(
    initialize=lista_loja_x_sku, doc="Lista de lojas por skus"
)

In [11]:
lista_niveis_ruptura = sheet_prob_ruptura["NIVEL_RUPTURA"].unique()
model.niveis_ruptura = pyo.Set(
    initialize=lista_niveis_ruptura, doc="Lista de niveis de ruptura"
)

In [12]:
lista_loja_x_sku_x_niveis_ruptura = [
    (row["LOJA"], row["SKU"], row["NIVEL_RUPTURA"]) for index, row in sheet_prob_ruptura.iterrows()
]
model.loja_x_sku_x_niveis_ruptura = pyo.Set(
    initialize=lista_loja_x_sku_x_niveis_ruptura, doc="Lista lojas por produto e niveis de ruptura"
)

### Parâmetros

In [13]:
def dataframe_to_dict(df, index_cols, value_col):
    all_cols = index_cols.copy()
    all_cols.append(value_col)

    return df[all_cols].set_index(index_cols).to_dict()[value_col]

In [14]:
disponibilidade_dict = dataframe_to_dict(sheet_disponibilidade_sku, index_cols=["SKU"], value_col="QUANTIDADE_DISPONIVEL")

model.disponibilidade = pyo.Param(
    model.skus,
    initialize=disponibilidade_dict,
    within=pyo.NonNegativeReals,
    doc="disponibilidade de skus",
)

In [15]:
capacidade_maxima_dict = dataframe_to_dict(
    sheet_loja_x_sku, index_cols=["LOJA", "SKU"], value_col="CAPACIDADE_MAXIMA"
)

model.capacidade_maxima = pyo.Param(
    model.lojas,
    model.skus,
    initialize=capacidade_maxima_dict,
    within=pyo.NonNegativeReals,
    doc="capacidade maxima da loja",
)

In [16]:
prob_min_aceitavel_dict = dataframe_to_dict(
    sheet_loja_x_sku, index_cols=["LOJA", "SKU"], value_col="PROBABILIDADE_ACEITAVEL"
)

model.prob_min_aceitavel = pyo.Param(
    model.lojas,
    model.skus,
    initialize=prob_min_aceitavel_dict,
    within=pyo.NonNegativeReals,
    doc="probabilidade minima aceitavel para nao ruptura",
)

In [17]:
nivel_estoque_dict = dataframe_to_dict(
    sheet_prob_ruptura,
    index_cols=["LOJA", "SKU", "NIVEL_RUPTURA"],
    value_col="QUANTIDADE_ESTOCADA",
)

model.nivel_estoque = pyo.Param(
    model.loja_x_sku_x_niveis_ruptura,
    initialize=nivel_estoque_dict,
    within=pyo.NonNegativeReals,
    doc="niveis de estoque e ruptura associada",
)

In [18]:
prob_ruptura_sku_dict = dataframe_to_dict(
    sheet_prob_ruptura,
    index_cols=["LOJA", "SKU", "NIVEL_RUPTURA"],
    value_col="PROBABILIDADE_DE_RUPTURA",
)

model.prob_ruptura_sku = pyo.Param(
    model.loja_x_sku_x_niveis_ruptura,
    initialize=prob_ruptura_sku_dict,
    within=pyo.NonNegativeReals,
    doc="prob ruptura do produto por nivel de estoque",
)

### Variáveis

In [19]:
model.QTD_SKU = pyo.Var(
    model.loja_x_sku,
    within=pyo.Integers,
    doc="variavel que define a quantidade de sku no estoque",
)

In [20]:
model.RUPTURA_SKU = pyo.Var(
    model.loja_x_sku,
    within=pyo.NonNegativeReals,
    doc="variavel que indica o nivel de ruptura do sku",
)

In [21]:
model.RUPTURA_LOJA_SKU = pyo.Var(
    model.loja_x_sku,
    within=pyo.Binary,
    doc="variavel binaria que indica se houve ruptura do produto na loja",
)

In [22]:
model.RUPTURA_LOJA = pyo.Var(
    model.lojas, within=pyo.Binary, doc="variavel binaria que indica se houve ruptura na loja"
)

In [23]:
model.NIVEL_SKU = pyo.Var(
    model.loja_x_sku_x_niveis_ruptura,
    within=pyo.Binary,
    doc="variavel que indica o nivel do estoque",
)

In [24]:
model.LAMBDA = pyo.Var(
    model.loja_x_sku_x_niveis_ruptura,
    within=pyo.NonNegativeReals,
    doc="variavel que auxilia na linearizacao por partes",
)

### Restrições

Respeita a capacidade do estoque de cada loja (redundante pois os niveis garantem que não ultrapassa a capacidade do estoque)

In [25]:
def _estoque_maximo(model, loja, sku):
    eq = (
        sum(
            model.NIVEL_SKU[(loja, sku, nivel_ruptura)]
            * model.nivel_estoque[(loja, sku, nivel_ruptura)]
            for nivel_ruptura in model.niveis_ruptura
        )
        <= model.capacidade_maxima[(loja, sku)]
    )
    return eq


model.estoque_maximo = pyo.Constraint(model.loja_x_sku, rule=_estoque_maximo)

calcula o estoque de acordo com o nível selecionado

In [26]:
def _quantidade_sku(model, loja, sku):
    eq = (
        sum(
            model.LAMBDA[(loja, sku, nivel_ruptura)]
            * model.nivel_estoque[(loja, sku, nivel_ruptura)]
            for nivel_ruptura in model.niveis_ruptura
        )
        == model.QTD_SKU[(loja, sku)]
    )
    return eq


model.quantidade_sku = pyo.Constraint(model.loja_x_sku, rule=_quantidade_sku)

calcula o 'limite superior' da ruptura, considerando ruptura aceitavel

In [27]:
def _ruptura_sku(model, loja, sku):
    eq = (
        sum(
            model.LAMBDA[(loja, sku, nivel_ruptura)]
            * model.prob_ruptura_sku[(loja, sku, nivel_ruptura)]
            for nivel_ruptura in model.niveis_ruptura
        )
        == model.RUPTURA_SKU[(loja, sku)]
    )
    return eq


model.ruptura_sku = pyo.Constraint(model.loja_x_sku, rule=_ruptura_sku)

respeita a disponibilidade de produtos

In [28]:
def _limite_disponibilidade(model, sku):
    eq = (
        sum(
            model.QTD_SKU[(loja, sku)]
            for (loja, _sku) in model.loja_x_sku
            if (_sku == sku)
        )
        <= model.disponibilidade[sku]
    )
    return eq


model.availability_limit = pyo.Constraint(model.skus, rule=_limite_disponibilidade)

no maximo um nível por loja e produto

In [29]:
def _max_nivel(model, loja, sku):
    eq = (
        sum(
            model.NIVEL_SKU[(loja, sku, nivel_ruptura)]
            for nivel_ruptura in model.niveis_ruptura
        )
        == 1
    )
    return eq


model.max_nivel = pyo.Constraint(model.loja_x_sku, rule=_max_nivel)

In [30]:
def _max_lambda(model, loja, sku):
    eq = (
        sum(
            model.LAMBDA[(loja, sku, nivel_ruptura)]
            for nivel_ruptura in model.niveis_ruptura
        )
        == 1
    )
    return eq


model.max_lambda = pyo.Constraint(model.loja_x_sku, rule=_max_lambda)

força 1 nivel em cada sku

In [31]:
def _nivel_minimo(model, loja, sku, nivel_ruptura):
    if nivel_ruptura == 1:
        eq = (
            model.LAMBDA[(loja, sku, nivel_ruptura)]
            <= model.NIVEL_SKU[(loja, sku, nivel_ruptura)]
        )
        return eq
    elif nivel_ruptura == 6:
        eq = (
            model.LAMBDA[(loja, sku, nivel_ruptura)]
            <= model.NIVEL_SKU[(loja, sku, nivel_ruptura-1)]
        )
        return eq
    else:
        eq = (
            model.LAMBDA[(loja, sku, nivel_ruptura)]
            <= model.NIVEL_SKU[(loja, sku, nivel_ruptura)]
            + model.NIVEL_SKU[(loja, sku, nivel_ruptura-1)]
        )
        return eq

model.min_level = pyo.Constraint(model.loja_x_sku_x_niveis_ruptura, rule=_nivel_minimo)

verifica se houve ruptura

In [32]:
def _houve_ruptura_sku(model, loja, sku):
    eq = (
        model.RUPTURA_SKU[(loja, sku)]
        - model.prob_min_aceitavel[(loja, sku)]
        <= model.RUPTURA_LOJA_SKU[(loja, sku)]
    )
    return eq

model.houve_ruptura_sku = pyo.Constraint(model.loja_x_sku, rule=_houve_ruptura_sku)

In [33]:
def _houve_ruptura_loja(model, loja, sku):
    eq = model.RUPTURA_LOJA[loja] >= model.RUPTURA_LOJA_SKU[(loja, sku)]
    return eq


model.houve_ruptura_loja = pyo.Constraint(model.loja_x_sku, rule=_houve_ruptura_loja)

### Função Objetivo

In [34]:
def _fo(model):
    #value = sum(model.RUPTURA_LOJA_SKU[(loja, sku)] for (loja, sku) in model.loja_x_sku)
    value = sum(model.RUPTURA_LOJA[loja] for loja in model.lojas)
    return value


model.fo = pyo.Objective(rule=_fo)

In [35]:
model.write("smart_execution_model.lp", io_options={'symbolic_solver_labels': True})

('smart_execution_model.lp', 2170575486872)

### Solve

In [36]:
opt = pyo.SolverFactory('glpk', executable='..\\glpk-4.65\\w64\\glpsol.exe')
results = opt.solve(model)

In [37]:
results.write(num=1)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3.0
  Upper bound: 3.0
  Number of objectives: 1
  Number of constraints: 185
  Number of variables: 219
  Number of nonzeros: 743
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 955
      Number of created subproblems: 955
  Error rc: 0
  Time: 0.7225282192230225
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

### Pós-processamento

In [38]:
def resultados_dataframe(var, index, column_names, csv_file=None):
    data = []
    n = len(column_names)
    for p in index:
        row = {}
        for i in range(n):
            row[column_names[i].lower()] = p[i]
        row["value"] = var[p].value
        data.append(row)
    df = pd.DataFrame(data)
    df.to_csv("resultados/" + csv_file, index=False)
    return df

In [39]:
df_qtd_sku = resultados_dataframe(
    model.QTD_SKU,
    index=model.loja_x_sku,
    column_names=["LOJA", "SKU"],
    csv_file="QTD_SKU.csv",
)
df_qtd_sku

,loja,sku,value
0,A,Mimmo Plus Suave,53.0
1,C,Mimmo Plus Suave,59.0
2,D,Mimmo Plus Suave,10.0
3,E,Mimmo Plus Suave,55.0
4,F,Mimmo Plus Suave,30.0
5,G,Mimmo Plus Suave,33.0
6,H,Mimmo Plus Suave,8.0
7,A,Report Premium,37.0
8,B,Report Premium,3.0
9,C,Report Premium,41.0


In [40]:
df_ruptura = resultados_dataframe(
    model.RUPTURA_SKU,
    index=model.loja_x_sku,
    column_names=["LOJA", "SKU"],
    csv_file="RUPTURA_SKU.csv",
)
df_ruptura

,loja,sku,value
0,A,Mimmo Plus Suave,0.040
1,C,Mimmo Plus Suave,0.396
2,D,Mimmo Plus Suave,0.700
3,E,Mimmo Plus Suave,0.080
4,F,Mimmo Plus Suave,0.300
5,G,Mimmo Plus Suave,0.040
6,H,Mimmo Plus Suave,1.000
7,A,Report Premium,0.010
8,B,Report Premium,0.960
9,C,Report Premium,0.372


In [41]:
df_houve_ruptura_loja_sku = resultados_dataframe(
    model.RUPTURA_LOJA_SKU,
    index=model.loja_x_sku,
    column_names=["LOJA", "SKU"],
    csv_file="RUPTURA_LOJA_SKU.csv",
)

df_houve_ruptura_loja_sku.loc[
    df_houve_ruptura_loja_sku.value==1
]

,loja,sku,value
2,D,Mimmo Plus Suave,1.0
6,H,Mimmo Plus Suave,1.0
8,B,Report Premium,1.0
10,D,Report Premium,1.0
13,H,Report Premium,1.0


In [42]:
df_ruptura_loja = resultados_dataframe(
    model.RUPTURA_LOJA,
    index=model.lojas,
    column_names=["LOJA"],
    csv_file="RUPTURA_LOJA.csv",
)
df_ruptura_loja.loc[
    df_ruptura_loja.value>0
] 

,loja,value
1,B,1.0
3,D,1.0
7,H,1.0


In [43]:
df_niveis_sku = resultados_dataframe(
        model.NIVEL_SKU,
    index=model.loja_x_sku_x_niveis_ruptura,
    column_names=["LOJA", "SKU", "NIVEL_RUPTURA"],
    csv_file="NIVEL_SKU.csv",
)

df_niveis_sku.loc[
    df_niveis_sku.value > 0
]

,loja,sku,nivel_ruptura,value
4,A,Mimmo Plus Suave,5,1.0
8,C,Mimmo Plus Suave,3,1.0
14,D,Mimmo Plus Suave,3,1.0
22,E,Mimmo Plus Suave,5,1.0
27,F,Mimmo Plus Suave,4,1.0
34,G,Mimmo Plus Suave,5,1.0
36,H,Mimmo Plus Suave,1,1.0
46,A,Report Premium,5,1.0
48,B,Report Premium,1,1.0
57,C,Report Premium,4,1.0


In [44]:
df_lambda = resultados_dataframe(
    model.LAMBDA,
    index=model.loja_x_sku_x_niveis_ruptura,
    column_names=["LOJA", "SKU", "NIVEL_RUPTURA"],
    csv_file="LAMBDA.csv",
)

df_lambda.loc[
    df_lambda.value > 0
]

,loja,sku,nivel_ruptura,value
4,A,Mimmo Plus Suave,5,5.617296e-33
5,A,Mimmo Plus Suave,6,1.000000e+00
8,C,Mimmo Plus Suave,3,5.000000e-02
9,C,Mimmo Plus Suave,4,9.500000e-01
14,D,Mimmo Plus Suave,3,1.000000e+00
23,E,Mimmo Plus Suave,6,1.000000e+00
27,F,Mimmo Plus Suave,4,1.000000e+00
34,G,Mimmo Plus Suave,5,7.401487e-18
35,G,Mimmo Plus Suave,6,1.000000e+00
36,H,Mimmo Plus Suave,1,1.000000e+00
